# Энграммная языковая модель

Напишем класс для нашей языковой модели, в котором будет реализован end-to-end пайплайн: чтение текстовых файлов, препроцессинг, подсчёт частотностей энграмм, построение матрицы вероятностей переходов, случайная генерация текста.

In [1]:
import numpy as np
from typing import *
from string import punctuation
from scipy.special import softmax
from nltk.tokenize import sent_tokenize
from collections import Counter, defaultdict

punctuation += '«»–—' # Расширяем строку punctuation кавычками-ёлочками и коротким и длинным тире.

class NgramLanguageModel:
    def __init__(self, path: str, n: int = 2) -> None:
        if n < 2:
            # Строго говоря, может, но униграммные модели в этом ноутбуке я решил не рассматривать.
            raise Exception('Параметр N энграммной языковой модели не может быть меньше 2!')
        self.n = n
        with open(path, encoding='utf-8') as f:
            self.text = f.read()
        self.sentences = map(lambda sentence: ['<START>'] * (self.n - 1) + \
                             [x for x in self.tokenize(sentence) if x] + ['<END>'],
                             sent_tokenize(self.text))
        self.token2id = {}
        if self.n > 2:
            self.ngram2id = {}
        self.ngrams = [Counter(), Counter()]
        self.get_ngram_counts()
        self.vocab_size = len(self.token2id)
        self.id2token = {value: key for key, value in self.token2id.items()}
        self.matrix = np.zeros((len(self.ngrams[0]), self.vocab_size))
        if self.n > 2:
            self.id2ngram = {value: key for key, value in self.ngram2id.items()}
        self.populate_matrix()
    
    def get_ngram_counts(self) -> None:
        if self.n == 2:
            for sentence in self.sentences:
                self.ngrams[0].update(sentence)
                self.ngrams[1].update(list(self.ngrammer(sentence, 2)))
                self.update_token2id(sentence)
        if self.n > 2:
            for sentence in self.sentences:
                ngrams = list(self.ngrammer(sentence, self.n-1))
                self.ngrams[0].update(ngrams)
                self.ngrams[1].update(list(self.ngrammer(sentence, self.n)))
                self.update_token2id(sentence)
                self.update_ngram2id(ngrams)
    
    def update_token2id(self, tokens: List[str]) -> None:
        for token in tokens:
            if token not in self.token2id:
                self.token2id[token] = len(self.token2id)
    
    def update_ngram2id(self, ngrams: List[str]) -> None:
        for ngram in ngrams:
            if ngram not in self.ngram2id:
                self.ngram2id[ngram] = len(self.ngram2id)
    
    def populate_matrix(self):
        if self.n == 2:
            for ngram, count in self.ngrams[1].items():
                src, dest = ngram.split()
                self.matrix[self.token2id[src], self.token2id[dest]] = \
                    count / self.ngrams[0][src]
        if self.n > 2:
            for ngram, count in self.ngrams[1].items():
                ngram_splitted = ngram.split()
                src = ' '.join(ngram_splitted[:-1])
                dest = ngram_splitted[-1]
                self.matrix[self.ngram2id[src], self.token2id[dest]] = \
                    count / self.ngrams[0][src]

    def generate_random_text(self, n: int = 100,
                             random_state: int = None) -> Generator[str, None, None]:
        if self.n == 2:
            current_idx = self.token2id['<START>']
            for i in range(n):
                if random_state:
                    np.random.seed(random_state)
                chosen = np.random.choice(self.vocab_size, p=self.matrix[current_idx])
                yield self.id2token[chosen]
                if chosen == self.token2id['<END>']:
                    chosen = self.token2id['<START>']
                current_idx = chosen
        if self.n > 2:
            current_idx = self.ngram2id[' '.join(['<START>'] * (self.n-1))]
            for i in range(n):
                if random_state:
                    np.random.seed(random_state)
                chosen = np.random.choice(self.vocab_size, p=self.matrix[current_idx])
                yield self.id2token[chosen]
                if chosen == self.token2id['<END>']:
                    current_idx = self.ngram2id[' '.join(['<START>'] * (self.n-1))]
                else:
                    current_idx = self.ngram2id[' '.join(self.id2ngram[current_idx].split()[1:] +
                                                         [self.id2token[chosen]])]
            
    @staticmethod
    def tokenize(sentence: str) -> map:
        return map(lambda x: x.strip(punctuation).replace('ё', 'е'),
                   sentence.strip().lower().split())
    
    @staticmethod
    def ngrammer(sentence: List[str], n: int) -> Generator[str, None, None]:
        for i in range(len(sentence)-n+1):
            yield ' '.join(sentence[i: i+n])

# Биграммная языковая модель

In [2]:
tolstoy2 = NgramLanguageModel('anna_karenina.txt', 2)
dostoevsky2 = NgramLanguageModel('besy_dostoevsky.txt', 2)

In [3]:
print(' '.join([x for x in tolstoy2.generate_random_text()]).replace('<END>', '\n'))

тяга была так сказывает ото ржей не найдете ли 
 кити не будет всегда чувствую как он понимает говорила она мужу с удобством без отдыха и ждали 
 говорят о том положении человека та вода 
 я покупаю сама княгиня тверская до сих пор пока уйдет с присоединившимся к нему точно так как не должна была счастлива буду избегать ее речь дорожа его жены чтобы не молчать и которой упоминалось о том как это со свернутыми набок голову при величайшем напряжении сил чувствовала всю эту осоку 
 нет я завидую 
 и рада что жена в мир для добра который


In [4]:
print(' '.join([x for x in dostoevsky2.generate_random_text()]).replace('<END>', '\n'))

да марья тимофеевна в доме и от одной радости голосом знаешь ли 
 еще в феврале когда девочке было ей расписывать про себя к публике начал возиться с патетическою иронией заметил липутин вскочил с самого святого фр 
 когда бедная заезжая дама да зачем 
 наши спины и так как же окруженною выбрала направление и к черту этих господ 
 у господина средних лет 
 кого ты ставрогину только то что… за сумасшедшего во главе юлию михайловну 
 в грустном настроении духа 
 в буфет которым он тогда вымирают народы и до свидания в него вынул из темной каморки еще


# Триграммная языковая модель

In [5]:
tolstoy3 = NgramLanguageModel('anna_karenina.txt', 3)
dostoevsky3 = NgramLanguageModel('besy_dostoevsky.txt', 3)

In [6]:
print(' '.join([x for x in tolstoy3.generate_random_text()]).replace('<END>', '\n'))

зачем я здесь стоя в холодке вновь покрытой риги с необсыпавшимся еще пахучим листом лещинового решетника прижатого к облупленным свежим осиновым слегам соломенной крыши левин глядел то сквозь открытые ворота в которых ныне находятся инородцы и в-четвертых наконец чтобы было столько неудач и столько враждебных отношений между им и смеется 
 он был здесь в москве в первый раз в сенях заинтересовался им 
 это несчастие совершившимся фактом и стараюсь помочь и ей не во мне много хорошего 
 они не думают о женихе и заняты и лишь он один теперь с варенькой… кажется что-то есть… может быть скучно на


In [7]:
print(' '.join([x for x in dostoevsky3.generate_random_text()]).replace('<END>', '\n'))

я положительно знаю что мне наскучила жизнь и за три дай просмотреть а то пожалуй и не сумею умереть кстати сегодня же я буду жить 
 пошли очень скоро шептать но я только ищу причины почему люди не любили народа не страдали за него более чем приносили пользы их изгоняют или казнят 
 тогда я начинал его ненавидеть… равно как и прежде хотел выйти однажды из службы давно уже на деле вникнуть лично и всегда… будем надеяться что и хмель соскочил 
 не беспокойтесь я сама пойду что бы то ни было но отставил чашку на стол 
 au diable


Предсказания триграммной языковой чуть больше похожи на связный текст. Возможно, если мы и дальше будем увеличивать длину энграмм, качество порождаемого текста будет расти, однако всё острее будет стоять проблема количества обучающих данных. Чем более длинный контекст мы хотим учитывать, тем больше данных для обучения нам нужно. Кроме того, по-хорошему, подсчитываемые корпусные вероятности надо бы сглаживать, например, функцией softmax (на этапе случайного выбора следующего токена), чтобы порождаемый текст был меньше похож на склеенные кусочки предложений оригинала, однако это создаёт новую проблему: получающиеся таким образом энграммы отсутствуют в корпусе. Методы борьбы с этой проблемой находятся вне скоупа этого домашнего задания.